#### Setting up environment

In [103]:
# importing modules

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import datetime
import ipywidgets as widgets

from docx import Document
from docx.shared import Inches, Length, Pt, Cm
from docx.enum.table import WD_ROW_HEIGHT_RULE
from docx.text.run import Font, Run
from docx.dml.color import ColorFormat
from docx.enum.dml import MSO_THEME_COLOR

In [95]:
# defining functions

def divide_chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n]
        
def delete_paragraph(paragraph):
    p = paragraph._element
    p.getparent().remove(p)
    p._p = p._element = None

#### Importing

In [96]:
# setting location, name, and codebook name for the XLS form

location = r'C:\Users\kitsc\Documents\Pragati Abhiyan\ragi_2019\forms\archive'
file = r'\t1_landnursery.xlsx'
codebook_file = r'\t1_landnursery.docx'

In [97]:
# importing files
survey = pd.read_excel(location + file, sheet_name="survey")
choices = pd.read_excel(location + file, sheet_name="choices")
settings = pd.read_excel(location + file, sheet_name="settings")
title = 'Codebook: ' + settings['form_title'][0]

# removing Markdown text
survey['label::English'] = survey['label::English'].str.replace(r"\<.*\"\>","", 1)
survey['label::English'] = survey['label::English'].str.replace(r'\<\/span\>',"")
# removing unnecessary lines
questions = survey.loc[~survey['type'].isin(['end group', 'username', 'simserial', 'calculate'])]

#### Creating document

In [98]:
# generating document and settings

document = Document()

margin = 1
for section in document.sections:
    section.top_margin = Cm(margin)
    section.bottom_margin = Cm(margin)
    section.left_margin = Cm(margin)
    section.right_margin = Cm(margin)
    
# adding preferred title
document.add_heading(title, 0)

# adding date and time
document.add_paragraph().add_run('This file was automatically generated on ' + 
                       datetime.datetime.now().strftime("%d-%b-%Y (%H:%M)")).italic = True

style = document.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(9)

#### Adding questionnaire items

In [99]:

sc = gc = qc = 0 # numbering counters

for i in questions.index:
    
    # settings
    p = document.add_paragraph()
    p.paragraph_format.space_before = Pt(12)
    p.paragraph_format.space_after = Pt(2)
    style = document.styles['Table Grid']
    q = questions['type'][i]
    q_type = questions['type'][i].split(' ', 1)[0]
    q_text = questions['label::English'][i]
    q_hint = questions['hint::English'][i]
    q_name = questions['name'][i]
    q_relevant = questions['relevant'][i]
    q_constraint = questions['constraint_message::English'][i]
    
    # REPEAT HEADINGS
    if q in ('begin repeat'):
        p.add_run('Begin repeated questions.').italic=True
        continue
    elif q in ('end repeat'):
        p.add_run('End repeated questions.').italic=True
        continue
    
    elif str(q_text) == 'nan':
        continue
        
    # GROUP HEADINGS
    elif q in ('begin group'): 

        # section headings
        if 's' in q_name and 'p' not in q_name:
            sc += 1
            gc = qc = 1 
            run = p.add_run(str(sc) + " " + q_text)
            run.font.size = Pt(20)
            run.font.color.theme_color = MSO_THEME_COLOR.ACCENT_4
            continue

        # part headings
        elif 'p' in q_name and 'g' not in q_name: 
            gc += 1
            qc = 1
            run = p.add_run(str(sc) + "." + str(gc) + " " + q_text)
            run.font.size = Pt(16)
            run.font.color.theme_color = MSO_THEME_COLOR.ACCENT_5
            continue

        # sub group headings
        elif 'g' in q_name:
            run = p.add_run(q_text)
            run.font.size = Pt(13)
            run.font.color.theme_color = MSO_THEME_COLOR.ACCENT_6
            continue

        else:
            continue

    # NOTES
    elif q in ('note'):
        p.add_run(q_text).bold = True
        continue

    # QUESTIONS
    else:
        qc += 1
        
        # add question number, variable name, variable label, hint
        p.add_run(str(sc) + "." + str(gc) + "." + str(qc) + " ").bold = True
        p.add_run(q_name + ", " + q_type + ": ").italic = True
        p.add_run(q_text).bold = True
        if str(q_hint) != 'nan':
            p.add_run(" Hint: " + q_hint).italic = True
            
        # add constraint, relevance
        if str(q_constraint) != 'nan':
            p1 = document.add_paragraph()
            p1.add_run('Constraint: ' + str(q_constraint)).font.size = Pt(6)
            p1.paragraph_format.space_before = Pt(2)
            p1.paragraph_format.space_after = Pt(2)
        if str(q_relevant) != 'nan':
            p2 = document.add_paragraph()
            p2.add_run('Relevance: ' + str(q_relevant)).font.size = Pt(6)
            p2.paragraph_format.space_before = Pt(2)
            p2.paragraph_format.space_after = Pt(2)
                        
        # text, numerical and location questions
        if q in ('text', 'geopoint', 'int', 'decimal', 'integer'):
            continue

        # select questions
        if ('select_multiple' in q) or ('select_one' in q):

            # set style and choice list header
            t = document.add_table(rows=0, cols=6)
            t.style = style

            # build choice list
            list_name = q.replace("select_one ", "").replace("select_multiple ", "")
            choices_list = choices.loc[choices['list_name'].isin([list_name])]
            choice_text = choices_list['name'].map(str) + "  " + choices_list['label::English'] \
                if choices_list['list_name'].any() not in ('villages', 'gps', 'regions') else choices_list['label::English']
            x = list(divide_chunks(list(choice_text), 6))
            for row in x: # get identical-sized lists
                for i in range(1,6):
                    if len(row) == i:
                        row.append('')
            for row in x: # add each list as a table row
                row_cells = t.add_row().cells
                for i, txt in enumerate(row):
                    row_cells[i].text = txt 
            
            # format choice list table
            for row in t.rows:
                for cell in row.cells:
                    paragraphs = cell.paragraphs
                    for paragraph in paragraphs:
                        for run in paragraph.runs:
                            run.font.size= Pt(6)

# remove whitespace
for paragraph in document.paragraphs:
    if len(paragraph.text)==0:
        delete_paragraph(paragraph)
        
document.save(location + codebook_file)

In [109]:
# UI
slider = widgets.IntSlider(min=10, max=100, value=10)
label = widgets.Label(value='Select number of games')
button = widgets.Button(description='Start Simulation', button_style='info', tooltip='Start Game')
progress = widgets.IntProgress(description='Progress:')
label2 = widgets.Label()
# Interactions
button.on_click(print('boo'))
# UI Layout
top_box = widgets.HBox([label, slider])
down_box = widgets.HBox([button, progress, label2])
ui = widgets.VBox([top_box, down_box])
display.display(ui)

boo


AttributeError: 'function' object has no attribute 'display'